### Getting Started Cohere

In [1]:
import cohere
api_key = '<your-api-key>'

co = cohere.Client(api_key)
co.check_api_key()

{'valid': True}

#### Co.Generate

In [2]:
res_generate = co.generate(
    prompt='Hello, how'
)

print(res_generate)

[cohere.Generation {
	id: fa077155-e450-46ff-869b-225a5796f31a
	prompt: Hello, how
	text:  Hello, how are you doing today?


	likelihood: None
	finish_reason: None
	token_likelihoods: None
}]


Nightly 版を使用し、最大 50 トークン含むテキストを 3 つ生成する

In [3]:
res_generate = co.generate(
    prompt='Hello, how',
    model='command-nightly',
    num_generations=3,
    max_tokens=50
)

print(res_generate)

[cohere.Generation {
	id: 7909718f-868d-440d-a533-bd41e51bb1ff
	prompt: Hello, how
	text:  Hello! How are you doing today?
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}, cohere.Generation {
	id: 78802579-5b4a-4ca5-a351-b26982342c7b
	prompt: Hello, how
	text:  Hello! I am a chatbot trained to have polite, helpful, inclusive conversations with people. How can I assist you today?
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}, cohere.Generation {
	id: 4ea214ed-59e9-436a-b44a-04eea492fb2b
	prompt: Hello, how
	text:  Hi! Is there anything I can help you with today?
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}]


最も可能性の高い上位 5 個(k=5)のトークンのみを生成対象として考慮し、生成におけるランダム性を最大(temperature=5.0)にする

In [4]:
res_generate = co.generate(
    prompt='Hello, how',
    model='command-nightly',
    num_generations=3,
    max_tokens=50,
    k=5,
    temperature=5.0
)

print(res_generate)

[cohere.Generation {
	id: 11d10cf3-8ed2-441d-b5ca-51ecebad9cf8
	prompt: Hello, how
	text:  I’m an A.I.-based assistant chatbot trained to be friendly and assist with questions you ask! Is there something you want me to do for your?
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}, cohere.Generation {
	id: d9903d59-cea1-4ee2-8150-5f2f81634313
	prompt: Hello, how
	text:  Hello to you as well ! How can I support you with today? 
--
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}, cohere.Generation {
	id: 6f05ee85-e340-4b59-bf65-fb4507526be9
	prompt: Hello, how
	text:  Hello. Is this your first visit? How can i direct your call?
	likelihood: None
	finish_reason: None
	token_likelihoods: None
}]


生成結果を JSON stream で受け取る（レスポンスの内容を少しずつレンダリングする UI にとって有益らしい）

In [5]:
res_generate_streaming = co.generate(
    prompt='Hello, how',
    stream=True
)

for index, token in enumerate(res_generate_streaming):
    print(f"{index}: {token}")

res_generate_streaming.texts

0: StreamingText(index=0, text=' Hello', is_finished=False)
1: StreamingText(index=0, text=',', is_finished=False)
2: StreamingText(index=0, text=' how', is_finished=False)
3: StreamingText(index=0, text=' can', is_finished=False)
4: StreamingText(index=0, text=' I', is_finished=False)
5: StreamingText(index=0, text=' help', is_finished=False)
6: StreamingText(index=0, text=' you', is_finished=False)
7: StreamingText(index=0, text='?', is_finished=False)
8: StreamingText(index=0, text='\n\n', is_finished=False)


[' Hello, how can I help you?\n\n']

#### Co.Embed

In [6]:
res_embed = co.embed(
    texts=['hello', 'world']
)

print(res_embed)

cohere.Embeddings {
	embeddings: [[1.6142578, 0.24841309, 0.5385742, -1.6630859, -0.27783203, 0.35888672, 1.3378906, -1.8261719, 0.89404297, 1.0791016, 1.0566406, 1.0664062, 0.20983887, -0.7558594, -0.29956055, -0.29248047, 0.23498535, -0.45947266, -0.7426758, -1.765625, 1.1699219, 2.0644531, 0.74902344, -1.6796875, 1.7685547, -0.7363281, 0.6376953, -1.6679688, -0.6621094, 1.2939453, 1.1904297, -2.3242188, 1.71875, 2.0117188, -0.46606445, 0.088134766, 1.6484375, -1.0292969, 0.07775879, -1.6503906, -0.035461426, 2.1660156, -1.2480469, -0.74072266, -0.53027344, -0.984375, -1.0039062, 0.47265625, -0.1574707, 1.984375, 0.72558594, 0.4885254, 0.25610352, -1.6113281, 1.7685547, -1.4824219, -3.4101562, -1.9023438, 0.47973633, -1.6972656, 0.34423828, 0.9121094, -0.046173096, 0.4482422, 0.828125, 1.7734375, -2.015625, -1.9423828, 0.63720703, -0.13000488, 2.5546875, -1.1972656, -1.0136719, -0.19714355, -0.43969727, 0.6591797, 0.46777344, -0.984375, -0.5629883, 0.014923096, 1.1787109, 1.4560547, 

軽量版モデル（model=embed-english-light-v2.0）を使ってみる

In [7]:
res_embed = co.embed(
    texts=['hello', 'world'],
    model='embed-english-light-v2.0'
)

print(res_embed)

cohere.Embeddings {
	embeddings: [[-0.16577148, -1.2109375, 0.54003906, -1.7148438, -1.5869141, 0.60839844, 0.6328125, 1.3974609, -0.49658203, -0.73046875, -1.796875, 1.5410156, 0.66064453, 0.9448242, -0.53515625, 0.24914551, 0.53222656, 0.23425293, 0.52685547, -1.3935547, 0.04095459, 0.8569336, -0.5620117, -0.42211914, 0.55371094, 3.5820312, 7.2890625, -1.2539062, 1.3583984, 0.12988281, -1.1660156, 0.124816895, 2.6835938, -0.57910156, -0.87353516, 1.3974609, 1.2519531, -0.41967773, 0.5546875, 1.5527344, 0.5703125, -0.3540039, 0.16296387, 0.64208984, -1.2402344, 0.053741455, -1.2460938, -2.2480469, -0.25830078, 1.6103516, -0.20581055, 0.3503418, -0.034454346, 0.44628906, 0.039520264, 0.8618164, 1.2226562, -0.5385742, 0.81152344, 1.4775391, 0.05709839, -0.09802246, -0.89160156, -0.51220703, -0.5917969, -1.546875, -0.6582031, -1.5498047, -1.2294922, -1.21875, -1.3652344, -0.73828125, 0.7001953, 0.6274414, -0.4609375, -0.9707031, 0.42089844, 0.7285156, 0.4169922, -1.4101562, 1.84375, 0.80

multilingual 対応のモデル（model=embed-multilingual-v2.0）を使ってみる

In [8]:
res_embed = co.embed(
    texts=['こんにちは', '世界'],
    model='embed-multilingual-v2.0'
)

print(res_embed)

cohere.Embeddings {
	embeddings: [[0.2590332, 0.41308594, 0.24279785, 0.30371094, 0.04647827, 0.1361084, 0.41357422, -0.40063477, 0.2553711, 0.17749023, -0.1899414, -0.041900635, 0.20141602, 0.43017578, -0.5878906, 0.18054199, 0.42333984, 0.010749817, -0.56640625, 0.1517334, 0.14282227, 0.36767578, 0.26953125, 0.1418457, 0.28051758, 0.1661377, -0.13293457, 0.23620605, 0.08703613, 0.36914062, 0.22180176, -0.3630371, -0.20117188, 0.09649658, -0.5727539, 0.21520996, 0.10333252, 0.12756348, -0.08782959, 0.20690918, -0.1665039, 0.26757812, 0.28564453, -0.052337646, 0.0055351257, 0.22521973, -0.072387695, 0.37353516, 0.057495117, 0.015136719, -0.13964844, 0.19909668, 0.07775879, -0.00907135, -0.7260742, 0.43041992, -0.030471802, 0.20129395, 0.13183594, -0.0066337585, 0.31103516, -0.23950195, -0.22070312, -0.0053520203, 0.2388916, 0.2298584, 0.0014648438, 0.02142334, 0.29345703, 0.09503174, -0.11920166, 0.18261719, 0.19067383, 0.12347412, -0.0012207031, 0.010719299, 0.14038086, -0.25219727, 0

#### Co.Classify

In [9]:
from cohere.responses.classify import Example

examples=[
  Example("Dermatologists don't like her!", "Spam"),
  Example("'Hello, open to this?'", "Spam"),
  Example("I need help please wire me $1000 right now", "Spam"),
  Example("Nice to know you ;)", "Spam"),
  Example("Please help me?", "Spam"),
  Example("Your parcel will be delivered today", "Not spam"),
  Example("Review changes to our Terms and Conditions", "Not spam"),
  Example("Weekly sync notes", "Not spam"),
  Example("'Re: Follow up from today's meeting'", "Not spam"),
  Example("Pre-read for tomorrow", "Not spam"),
]

inputs=[
  "Confirm your email address",
  "hey i need u to send some $",
]

res_classify = co.classify(
    inputs=inputs,
    examples=examples,
)

print(res_classify)

[Classification<prediction: "Not spam", confidence: 0.5661598, labels: {'Not spam': LabelPrediction(confidence=0.5661598), 'Spam': LabelPrediction(confidence=0.43384025)}>, Classification<prediction: "Spam", confidence: 0.9909811, labels: {'Not spam': LabelPrediction(confidence=0.009018883), 'Spam': LabelPrediction(confidence=0.9909811)}>]


軽量版モデル（model=embed-english-light-v2.0）で最大トークン長より長い入力を渡した際は、エラーが返却される（truncate=NONE）ようにする

In [10]:
inputs_max_exceeded=[
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
  "Confirm your email address",
  "hey i need u to send some $",
  "Please sign the receipt attached for the arrival of new office facilities.",
  "Attached is the paper concerning with the cancellation of your current credit card.",
  "The monthly financial statement is attached within the email.",
  "The audit report you inquired is attached in the mail. Please review and transfer it to the related department.",
  "There is a message for you from 01478719696, on 2016/08/23 18:26:30. You might want to check it when you get a chance.Thanks!",
  "Dear John, I am attaching the mortgage documents relating to your department. They need to be signed in urgent manner.",
  "This email/fax transmission is confidential and intended solely for the person or organisation to whom it is addressed. If you are not the intended recipient, you must not copy, distribute or disseminate the information, or take any action in reliance of it. Any views expressed in this message are those of the individual sender, except where the sender specifically states them to be the views of any organisation or employer. If you have received this message in error, do not open any attachment but please notify the sender (above) deleting this message from your system. For email transmissions please rely on your own virus check no responsibility is taken by the sender for any damage rising out of any bug or virus infection.",
  "Sent with Genius Scan for iOS. http://bit.ly/download-genius-scan",
  "Hey John, as you requested, attached is the paycheck for your next month?s salary in advance.",
  "I am sending you the flight tickets for your business conference abroad next month. Please see the attached and note the date and time.",
  "Attached is the bank transactions made from the company during last month. Please file these transactions into financial record.",
  "example.com SV9100 InMail",
  "Attached file is scanned image in PDF format. Use Acrobat(R)Reader(R) or Adobe(R)Reader(R) of Adobe Systems Incorporated to view the document. Adobe(R)Reader(R) can be downloaded from the following URL: Adobe, the Adobe logo, Acrobat, the Adobe PDF logo, and Reader are registered trademarks or trademarks of Adobe Systems Incorporated in the United States and other countries.",
  "Here is the travel expense sheet for your upcoming company field trip. Please write down the approximate costs in the attachment.",
  "Attached is the list of old office facilities that need to be replaced. Please copy the list into the purchase order form.",
  "We are sending you the credit card receipt from yesterday. Please match the card number and amount.",
  "Queries to: <scanner@example.com",
  "Please find our invoice attached.",
  "Hello! We are looking for employees working remotely. My name is Margaret, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $3000-$6000. If you are interested in this offer, please visit",
  "Please see the attached copy of the report.",
  "Attached is a pdf file containing items that have shipped. Please contact us if there are any questions or further assistance we can provide",
  "Please see the attached license and send it to the head office.",
  "Please find attached documents as requested.",
  "We are looking for employees working remotely. My name is Enid, I am the personnel manager of a large International company. Most of the work you can do from home, that is, at a distance. Salary is $2000-$5400. If you are interested in this offer, please visit",
  "Your item #123456789 has been sent to you by carrier. He will arrive to youon 23th of September, 2016 at noon.",
  "Dear John, we are very sorry to inform you that the item you requested is out of stock. Here is the list of items similar to the ones you requested. Please take a look and let us know if you would like to substitute with any of them.",
  "Thank you for your order! Please find your final sales receipt attached to this email. Your USPS Tracking Number is: 123456789. This order will ship tomorrow and you should be able to begin tracking tomorrow evening after it is picked up. If you have any questions or experience any problems, please let us know so we can assist you. Thanks again and enjoy! Thanks,",
  "Thanks for using electronic billing. Please find your document attached",
  "Dear John, I attached the clients’ accounts for your next operation. Please look through them and collect their data. I expect to hear from you soon.",
  "You are receiving this email because the company has assigned you as part of the approval team. Please review the attached proposal form and make your approval decision. If you have any problem regarding the submission, please contact Gonzalo.",
  "this is to inform you that your Debit Card is temporarily blocked as there were unknown transactions made today. We attached the scan of transactions. Please confirm whether you made these transactions.",
  "The letter is attached. Please review his concerns carefully and reply him as soon as possible. King regards,",
]

res_classify = co.classify(
    inputs=inputs_max_exceeded,
    examples=examples,
    model='embed-english-light-v2.0',
    truncate='END'
)

res_classify.classifications

CohereAPIError: invalid request: inputs cannot contain more than 96 elements, received 136

#### Co.Tokenize

In [11]:
res_tokenize = co.tokenize(
    text='tokenize me !D',
)

print(res_tokenize)

cohere.Tokens {
	tokens: [10002, 2261, 2012, 3362, 43]
	token_strings: ['token', 'ize', ' me', ' !', 'D']
	meta: {'api_version': {'version': '1'}}
}


Co.Detokenize

In [12]:
tokens = [10002, 2261, 2012, 3362, 43] # tokenize me !D

res_detokenize = co.detokenize(
    tokens=tokens
)

print(res_detokenize)

tokenize me !D


#### Co.Detect_language

In [13]:
res_detect_language = co.detect_language(
    texts=['Hello world', "'Здравствуй, Мир'", 'こんにちは世界', '世界你好', '안녕하세요']
)

res_detect_language

cohere.DetectLanguageResponse {
	results: [Language<language_code: "en", language_name: "English">, Language<language_code: "ru", language_name: "Russian">, Language<language_code: "ja", language_name: "Japanese">, Language<language_code: "zh", language_name: "Chinese">, Language<language_code: "ko", language_name: "Korean">]
	meta: {'api_version': {'version': '1'}}
}

#### Co.summarize

In [14]:
text = (
  "Ice cream is a sweetened frozen food typically eaten as a snack or dessert. "
  "It may be made from milk or cream and is flavoured with a sweetener, "
  "either sugar or an alternative, and a spice, such as cocoa or vanilla, "
  "or with fruit such as strawberries or peaches. "
  "It can also be made by whisking a flavored cream base and liquid nitrogen together. "
  "Food coloring is sometimes added, in addition to stabilizers. "
  "The mixture is cooled below the freezing point of water and stirred to incorporate air spaces "
  "and to prevent detectable ice crystals from forming. The result is a smooth, "
  "semi-solid foam that is solid at very low temperatures (below 2 °C or 35 °F). "
  "It becomes more malleable as its temperature increases.\n\n"
  "The meaning of the name \"ice cream\" varies from one country to another. "
  "In some countries, such as the United States, \"ice cream\" applies only to a specific variety, "
  "and most governments regulate the commercial use of the various terms according to the "
  "relative quantities of the main ingredients, notably the amount of cream. "
  "Products that do not meet the criteria to be called ice cream are sometimes labelled "
  "\"frozen dairy dessert\" instead. In other countries, such as Italy and Argentina, "
  "one word is used fo\r all variants. Analogues made from dairy alternatives, "
  "such as goat's or sheep's milk, or milk substitutes "
  "(e.g., soy, cashew, coconut, almond milk or tofu), are available for those who are "
  "lactose intolerant, allergic to dairy protein or vegan."
)

co_summarize = co.summarize(
    text=text
)

print(co_summarize)

SummarizeResponse(id='4ef8d546-b6bc-4e97-a3be-9d2f44c15607', summary='Ice cream is a sweetened frozen food typically eaten as a snack or dessert. It is made from milk or cream and is flavored with a sweetener and a spice or with fruit. It can also be made by whisking a flavored cream base and liquid nitrogen together. Food coloring and stabilizers may be added. It is cooled and stirred to incorporate air spaces and to prevent ice crystals from forming. It is solid at very low temperatures and becomes more malleable as its temperature increases. The meaning of the name "ice cream" varies from one country to another. Some countries regulate the commercial use of the various terms according to the relative quantities of the main ingredients. Dairy alternatives are available for those who are lactose intolerant, allergic to dairy protein or vegan.', meta={'api_version': {'version': '1'}})


長さを短く（length=short）し、箇条書き形式（format=bullets）で出力する

In [15]:
co_summarize = co.summarize(
    text=text,
    length='short',
    format='bullets'
)

print(co_summarize)

SummarizeResponse(id='1bca4ac5-8387-4d81-bef9-3fc3efdd01b4', summary='- Ice cream is a frozen dessert made from milk or cream and is flavoured with a sweetener and a spice or fruit.\n- It is available in many countries and can also be made from dairy alternatives.', meta={'api_version': {'version': '1'}})


軽量版のモデルを使い（model=command-light）、元の文章からの抽出度を高くし（extractiveness=high）、箇条書き形式で出力する

In [16]:
co_summarize = co.summarize(
    text=text,
    model='command-light',
    length='short',
    format='bullets',
    extractiveness='high'
)

print(co_summarize)

SummarizeResponse(id='e0cd6176-aa75-415d-ae2f-82cb424f1f32', summary="- Ice cream is a sweetened frozen food typically eaten as a snack or dessert.\n- It may be made from milk or cream and is flavored with a sweetener, either sugar or an alternative, and a spice, such as cocoa or vanilla.\n- It can also be made by whisking a flavored cream base and liquid nitrogen together.\n- Ice cream can be made from dairy alternatives, such as goat's or sheep's milk, or milk substitutes (e.g., soy, cashew, coconut, almond milk or tofu).", meta={'api_version': {'version': '1'}})


#### Co.Rerank

In [17]:
docs = [
    'Carson City is the capital city of the American state of Nevada.',
    'The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.',
    'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.',
    'Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.'
]

co_rerank = co.rerank(
    model='rerank-english-v2.0',
    query='What is the capital of the United States?',
    documents=docs
)

print(co_rerank)

[RerankResult<document['text']: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district., index: 2, relevance_score: 0.98005307>, RerankResult<document['text']: Capital punishment (the death penalty) has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states., index: 3, relevance_score: 0.27904198>, RerankResult<document['text']: Carson City is the capital city of the American state of Nevada., index: 0, relevance_score: 0.10194652>, RerankResult<document['text']: The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan., index: 1, relevance_score: 0.0721122>]


日本語で試してみる（model=rerank-multilingual-v2.0）

In [18]:
docs_ja = [
    'カーソンシティはアメリカ・ネバダ州の州都である',
    '北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。',
    ' ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である',
    '死刑制度はアメリカ合衆国が国である以前から存在する。2017年現在、死刑は50州のうち30州で合法である。'
]

co_rerank_ja = co.rerank(
    model='rerank-multilingual-v2.0',
    query='アメリカの首都は何ですか？',
    documents=docs_ja
)

print(co_rerank_ja)

[RerankResult<document['text']:  ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である, index: 2, relevance_score: 0.9998813>, RerankResult<document['text']: カーソンシティはアメリカ・ネバダ州の州都である, index: 0, relevance_score: 0.37903354>, RerankResult<document['text']: 北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。, index: 1, relevance_score: 0.23899394>, RerankResult<document['text']: 死刑制度はアメリカ合衆国が国である以前から存在する。2017年現在、死刑は50州のうち30州で合法である。, index: 3, relevance_score: 0.0007014083>]


日本語（model=rerank-multilingual-v2.0）で最も相関の高いもののみ（top_n=1）を返却する。

In [19]:
docs_ja = [
    'カーソンシティはアメリカ・ネバダ州の州都である',
    '北マリアナ諸島連邦は太平洋に浮かぶ島々である。首都はサイパンである。',
    ' ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である',
    '死刑制度はアメリカ合衆国が国である以前から存在する。2017年現在、死刑は50州のうち30州で合法である。'
]

co_rerank_ja = co.rerank(
    model='rerank-multilingual-v2.0',
    query='アメリカの首都は何ですか？',
    documents=docs_ja,
    top_n=1,
)

print(co_rerank_ja)

[RerankResult<document['text']:  ワシントンD.C.（Washington, D.C.）は、アメリカ合衆国の首都である。連邦区である, index: 2, relevance_score: 0.9998813>]
